# Bayesian Moderated Mediation Analysis: A Step-by-Step Guide

**From Theory to Implementation in brms**

**Author:** Bryan Gibson  
**Date:** 2025

---

## Abstract

### Technical Abstract

This tutorial provides a comprehensive framework for conducting Bayesian moderated mediation analysis (first-stage moderation; Hayes Model 8) using the brms package in R. We demonstrate estimation of the Index of Moderated Mediation (IMM = $a_3 \times b$) through multivariate Bayesian regression with weakly informative priors. Monte Carlo simulation (500 replications per cell) evaluates estimator performance across sample sizes (N = 100-1000), effect magnitudes (IMM = 0-0.16), and missing data conditions (0-20% MCAR). Results confirm unbiased estimation (|bias| < 0.01), nominal coverage of 95% credible intervals (94-96%), and adequate power (≥80%) for medium effects (IMM = 0.10) at N ≥ 250. Four empirical demonstrations spanning social psychology, labor economics, communication, and clinical domains illustrate both successful detection (Garcia: IMM = 0.27 [0.07, 0.49]; SLID: IMM = 0.20 [0.12, 0.28]) and appropriate null findings (Tal-Or, FacialBurns). Direct comparison with frequentist bootstrapping shows convergent validity for adequately powered studies, with Bayesian advantages emerging for small samples (N < 100), mixed outcome families, and posterior probability inference. Complete annotated R code, diagnostic procedures, and reporting templates are provided.

**Keywords**: Bayesian inference, moderated mediation, conditional indirect effects, brms, MCMC, Index of Moderated Mediation, tutorial, R, prior specification

### Plain Language Summary

Sometimes the effect of one variable on another works through a middle step (mediation), and this middle step may be stronger or weaker depending on a third variable (moderation). Testing this "moderated mediation" is common in psychology but typically uses methods that don't work well with small samples or complex data.

This tutorial teaches you a Bayesian approach---a statistical method that directly tells you "there's a 95% chance the effect is between X and Y"---which is what most researchers actually want to know. We show you exactly how to do this in R using the brms package, with step-by-step code you can adapt.

We tested the method with computer simulations and found it works well: you need about 250 participants to reliably detect a medium-sized effect. We also applied it to four real datasets from different research areas. Two showed significant moderated mediation; two didn't---and we explain why this happened.

The tutorial includes templates for writing up your results, common mistakes to avoid, and guidance on when to use Bayesian versus traditional methods. All code is available online so you can apply these techniques to your own data.

---

# Introduction

This guide provides a comprehensive walkthrough for conducting Bayesian moderated mediation analysis using the `brms` package in R. Moderated mediation---also called conditional indirect effects or conditional process analysis---represents one of the most important analytical frameworks in contemporary psychological and organizational research (Hayes, 2018; Preacher et al., 2007). While frequentist methods for moderated mediation are well-established (Hayes, 2013; Preacher & Hayes, 2008), Bayesian approaches offer several distinct advantages that have been underutilized in applied research.

## Literature Review: Bayesian Approaches to Mediation

### Historical Development

The foundation of Bayesian mediation analysis was laid by Yuan and MacKinnon (2009), who demonstrated that Bayesian methods could recover indirect effects with greater precision than bootstrapping, particularly with small samples and non-normal data. Their work established that the posterior distribution of the indirect effect ($ab$) naturally captures the non-symmetric, potentially non-normal shape that characterizes products of coefficients.

Subsequent developments by Muthén and Asparouhov (2012, 2015) extended Bayesian mediation to structural equation modeling frameworks, introducing techniques for handling complex latent variable models with Bayesian estimation. They demonstrated particular advantages for models with small sample sizes where maximum likelihood estimation performs poorly. Wang and Preacher (2015) provided the first comprehensive treatment of Bayesian moderated mediation specifically, establishing the methodological foundation for the approach presented in this tutorial.

### Advantages of Bayesian Moderated Mediation

Several key advantages motivate the Bayesian approach:

1. **Proper uncertainty quantification**: The Index of Moderated Mediation (IMM = $a_3 \times b$) is a product of parameters with inherently non-normal sampling distribution. Bayesian methods directly compute the posterior distribution without asymptotic assumptions.

2. **Prior information integration**: Researchers can incorporate previous findings through informative priors, particularly valuable in replication studies or when testing theoretically constrained hypotheses.

3. **Interpretable inference**: The 95% credible interval directly states "there is a 95% probability the parameter lies within this range"---a more intuitive interpretation than frequentist confidence intervals.

4. **Small sample performance**: Regularizing priors improve estimation stability when sample sizes are limited, a common challenge in organizational and clinical research (McNeish, 2016; Smid et al., 2020). Miočević et al. (2017) demonstrated that Bayesian methods for mediation effect sizes show superior properties compared to frequentist alternatives.

5. **Model flexibility**: The `brms` package enables multivariate models with mixed response types (continuous mediators, binary outcomes), multilevel structures, and complex non-linear relationships.

### Current State of the Literature

Despite these advantages, Bayesian moderated mediation remains underrepresented in psychological methods literature. A review of recent publications reveals:

- Most tutorials focus on simple mediation (e.g., Nuijten et al., 2015)
- Moderated mediation guidance typically uses frequentist frameworks (e.g., Hayes & Rockwood, 2017)
- Few resources bridge the gap between Bayesian theory and practical implementation

This guide addresses that gap by providing step-by-step implementation with `brms`, empirical demonstrations across multiple domains, and direct comparison with frequentist alternatives.

## Structure of This Guide

Each section covers:

- **Conceptual explanation**: What the step does and why it matters
- **Code snippet**: How to implement it
- **Interpretation guide**: What to look for in results
- **Common pitfalls**: Troubleshooting and mistakes to avoid

## When to Use This Analysis

Moderated mediation (also called conditional indirect effects) is appropriate when:

1. You hypothesize that X affects Y through mediator M
2. The strength of this indirect effect depends on a moderator W
3. You want full posterior distributions rather than point estimates

## Running Example

Throughout this guide, we use:

- **X**: Broker Network Richness (BNR) - diversity of professional contacts
- **M**: Time to Meeting (TTM) - weeks until first investor meeting
- **Y**: Funding Success (FS) - binary outcome (0/1)
- **W**: Founder Preparedness (FP) - moderator variable

In [ ]:
# Setup - Load required packages
library(knitr)
library(kableExtra)

---

# Step 1: Conceptualize Your Model

## Conceptual Explanation

Before writing any code, clearly articulate your theoretical model. This involves:

1. **Identifying the causal pathway**: X → M → Y
2. **Specifying where moderation occurs**: First-stage (X→M), second-stage (M→Y), or both
3. **Stating directional hypotheses**: What signs do you expect?

### Types of Moderated Mediation

**First-Stage Moderation**: W moderates the X → M path  
**Second-Stage Moderation**: W moderates the M → Y path

## Code Snippet: Documenting Your Model

In [ ]:
# Document your theoretical model clearly
model_specification <- list(
  # Variables
  X = "Broker Network Richness (BNR)",
  M = "Time to Meeting (TTM)",
  Y = "Funding Success (FS)",
  W = "Founder Preparedness (FP)",

  # Moderation type
  moderation_type = "first-stage",  # W moderates X -> M

  # Hypotheses with expected directions
  hypotheses = list(
    H1 = list(
      path = "a (X -> M)",
      expected = "negative",
      rationale = "More diverse networks provide faster access to investors"
    ),
    H2 = list(
      path = "a × W interaction",
      expected = "negative",
      rationale = "Prepared founders leverage diverse networks more effectively"
    )
  ),

  # Control variables
  controls = c("Meeting Quality"),

  # Level-2 grouping (if multilevel)
  grouping = "Market"
)

print(model_specification)

## Interpretation Guide

Your theoretical model should answer:

| Question | Your Answer |
|----------|-------------|
| What is the hypothesized pathway? | X predicts M, which in turn predicts Y |
| Where does moderation occur? | First-stage: W affects the X → M path |
| What are the expected signs? | a < 0, b < 0, therefore indirect > 0 |
| Is there a direct effect? | Yes, c' represents the association not through M |

**Note on causal language**: The arrows in path diagrams (→) represent hypothesized directional relationships, not established causation. With observational data, we can only demonstrate associations consistent with the model; experimental manipulation or design-based identification is required to make causal claims.

## Common Pitfalls

**Pitfall 1: Confusing moderation types**

- **First-stage**: W moderates a path (X → M)
- **Second-stage**: W moderates b path (M → Y)
- **Dual moderation**: W moderates both (more complex, harder to interpret)

**Pitfall 2: Including moderation in wrong equation**

In [ ]:
# WRONG: This is dual moderation, not first-stage
# outcome ~ X * W + M  # X×W interaction in outcome

# CORRECT: First-stage only
# mediator ~ X * W     # Interaction only in mediator equation
# outcome ~ X + M      # No interaction in outcome

---

# Step 2: Prepare Data

## Conceptual Explanation

Data preparation ensures your model runs efficiently and coefficients are interpretable:

1. **Standardize continuous predictors**: Centers variables and puts them on same scale
2. **Handle missing data**: Listwise deletion or multiple imputation
3. **Check distributions**: Identify skewness, outliers, floor/ceiling effects
4. **Create interaction terms**: For lavaan (brms creates them automatically)

## Code Snippet

In [ ]:
library(tidyverse)

# Simulate example data
set.seed(42)
n <- 500

raw_data <- tibble(
  # Independent variable: Broker Network Richness (Blau Index, 0-0.8)
  BNR = rbeta(n, 2, 3) * 0.8,

  # Moderator: Founder Preparedness (0-100 scale)
  FP = rnorm(n, 60, 15) %>% pmax(0) %>% pmin(100),

  # Mediator: Time to Meeting (weeks, right-skewed)
  TTM = 4 + exp(2 - 3*BNR - 0.01*FP - 0.02*BNR*FP + rnorm(n, 0, 0.5)),

  # Outcome: Funding Success (binary)
  prob_success = plogis(-2 + 2*BNR - 0.1*TTM + 0.01*FP),
  FS = rbinom(n, 1, prob_success),

  # Grouping variable
  Market = sample(c("Atlanta", "Austin", "Boston", "Denver"), n, replace = TRUE)
)

# Check raw distributions
cat("=== Raw Data Summary ===\n")
summary(raw_data[, c("BNR", "FP", "TTM", "FS")])

In [ ]:
# Standardize predictors
model_data <- raw_data %>%
  mutate(
    # Standardize continuous predictors (mean=0, sd=1)
    BNR_z = scale(BNR)[,1],
    FP_z = scale(FP)[,1],

    # Keep mediator on original scale or standardize
    # (standardizing makes coefficients comparable)
    TTM_z = scale(TTM)[,1],

    # Create explicit interaction for lavaan
    BNR_FP = BNR_z * FP_z
  ) %>%
  # Remove missing data
  drop_na()

cat("\n=== Standardized Data Summary ===\n")
cat("N =", nrow(model_data), "\n")
cat("Funding Success rate:", round(mean(model_data$FS), 3), "\n")

In [ ]:
# Check distributions
library(patchwork)

p1 <- ggplot(model_data, aes(x = BNR_z)) +
  geom_histogram(bins = 30, fill = "steelblue", alpha = 0.7) +
  labs(title = "BNR (standardized)", x = NULL) +
  theme_minimal()

p2 <- ggplot(model_data, aes(x = TTM_z)) +
  geom_histogram(bins = 30, fill = "coral", alpha = 0.7) +
  labs(title = "TTM (standardized)", x = NULL) +
  theme_minimal()

p3 <- ggplot(model_data, aes(x = factor(FS))) +
  geom_bar(fill = "seagreen", alpha = 0.7) +
  labs(title = "Funding Success", x = NULL) +
  theme_minimal()

p1 + p2 + p3

## Interpretation Guide

After standardization, check:

| Metric | Ideal | Concern |
|--------|-------|--------|
| Mean of standardized vars | ~0 | If not 0, check calculation |
| SD of standardized vars | ~1 | If not 1, check calculation |
| Skewness | \|skew\| < 2 | Consider transformation |
| Missing data | < 5% | Consider imputation if higher |
| Outcome balance | 20-80% | Rare events need special handling |

## Common Pitfalls

**Pitfall 1: Forgetting to standardize**

Unstandardized predictors on different scales make:
- Coefficients hard to compare
- Priors harder to specify
- MCMC sampling less efficient

**Pitfall 2: Standardizing the outcome**

In [ ]:
# WRONG: Don't standardize binary outcomes
# FS_z = scale(FS)  # NO!

# CORRECT: Keep binary outcomes as 0/1
# FS = Funding_Success  # Already 0/1

**Pitfall 3: Creating interaction before standardizing**

In [ ]:
# WRONG: Creates correlation between main effect and interaction
# BNR_FP = BNR * FP
# BNR_z = scale(BNR)
# FP_z = scale(FP)

# CORRECT: Standardize first, then create interaction
# BNR_z = scale(BNR)
# FP_z = scale(FP)
# BNR_FP = BNR_z * FP_z  # Interaction of standardized terms

---

# Step 3: Specify Priors

## Conceptual Explanation

Bayesian analysis requires specifying prior distributions that encode your beliefs before seeing the data. For most applications, use **weakly informative priors** that:

1. Rule out implausible values
2. Don't overwhelm the data
3. Improve computational stability

### Prior Specification Strategy

For standardized predictors:

| Parameter Type | Recommended Prior | Rationale |
|---------------|-------------------|----------|
| Fixed effects (β) | Normal(0, 1.5) | Allows coefficients up to ±3 SD |
| Interaction terms | Normal(0, 1) | Tighter prior for stability |
| Random intercept SD | Exponential(1) | Half-normal, allows moderate variation |
| Random slope SD | Exponential(2) | Tighter, slopes vary less than intercepts |
| Residual SD | Exponential(1) | Standard choice for continuous outcomes |

## Code Snippet

In [ ]:
library(brms)

# Priors for multivariate model with different response families
priors_mediation <- c(
  # --- Mediator model (continuous: TTM) ---
  # Fixed effects
  prior(normal(0, 1.5), class = "b", resp = "TTMz"),
  # Interaction term (tighter for stability)
  prior(normal(0, 1), class = "b", coef = "BNR_z:FP_z", resp = "TTMz"),
  # Random effects SD
  prior(exponential(1), class = "sd", resp = "TTMz"),
  # Residual SD
  prior(exponential(1), class = "sigma", resp = "TTMz"),

  # --- Outcome model (binary: FS) ---
  # Fixed effects (on log-odds scale)
  prior(normal(0, 1.5), class = "b", resp = "FS"),
  # Random effects SD
  prior(exponential(1), class = "sd", resp = "FS")
  # Note: No sigma for Bernoulli family
)

# View prior specifications
print(priors_mediation)

### Visualizing Priors

In [ ]:
library(ggplot2)

# Visualize what these priors imply
x <- seq(-5, 5, length.out = 1000)

prior_viz <- tibble(
  x = x,
  `Normal(0, 1.5)` = dnorm(x, 0, 1.5),
  `Normal(0, 1)` = dnorm(x, 0, 1),
  `Normal(0, 2.5)` = dnorm(x, 0, 2.5)
) %>%
  pivot_longer(-x, names_to = "Prior", values_to = "Density")

ggplot(prior_viz, aes(x = x, y = Density, color = Prior)) +
  geom_line(linewidth = 1) +
  labs(title = "Prior Distributions for Fixed Effects",
       x = "Coefficient Value", y = "Density") +
  theme_minimal() +
  scale_color_brewer(palette = "Set1")

## Common Pitfalls

**Pitfall 1: Flat priors**

In [ ]:
# PROBLEMATIC: Flat priors can cause computational issues
# prior(uniform(-Inf, Inf), class = "b")  # NO!

# BETTER: Weakly informative
# prior(normal(0, 10), class = "b")  # Wide but proper

**Pitfall 2: Forgetting `resp` for multivariate models**

In [ ]:
# WRONG: brms doesn't know which equation this applies to
# prior(normal(0, 1.5), class = "b")  # Error!

# CORRECT: Specify response variable
# prior(normal(0, 1.5), class = "b", resp = "TTMz")
# prior(normal(0, 1.5), class = "b", resp = "FS")

**FATAL PITFALL: Priors assume standardized predictors!**

**This is critical**: All priors in this tutorial (and most `brms` tutorials) assume your predictors were standardized in Step 2. If you skip standardization, these priors become wildly inappropriate.

**Rule**: If you use `Normal(0, 1)` or `Normal(0, 1.5)` priors, your predictors **must** be z-scored. No exceptions.

---

# Step 4: Fit Models

## Conceptual Explanation

In `brms`, we fit mediator and outcome models jointly using multivariate syntax. This:

1. **Properly propagates uncertainty** from M to Y equation
2. **Allows correlated random effects** across equations (if desired)
3. **Enables easy extraction** of indirect effects from joint posterior

### Model Specification in brms

The key syntax elements:

- `bf()`: Defines a single equation (formula)
- `+`: Combines equations into multivariate model
- `family`: Specifies distribution (gaussian, bernoulli, etc.)
- `set_rescor(FALSE)`: Disables residual correlation estimation

**Why `set_rescor(FALSE)`?** When your mediator is continuous (Gaussian) but your outcome is binary (Bernoulli), the residual correlation is not mathematically well-defined. A Gaussian model has a continuous residual ($\epsilon \sim N(0, \sigma^2)$), but a Bernoulli model has no residual variance parameter---the variance is fully determined by the mean ($\text{Var}(Y) = p(1-p)$). Setting `rescor = FALSE` tells `brms` to skip this undefined parameter.

## Code Snippet

In [ ]:
library(brms)

# Fit the multivariate mediation model
# This may take 2-5 minutes depending on your computer

fit_mediation <- brm(
  # Mediator model: TTM ~ BNR * FP (first-stage moderation)
  bf(TTM_z ~ BNR_z * FP_z + (1 | Market)) +

  # Outcome model: FS ~ BNR + TTM (no interaction = first-stage only)
  bf(FS ~ BNR_z + FP_z + TTM_z + (1 | Market),
     family = bernoulli(link = "logit")) +

  # No residual correlation (different families)
  set_rescor(FALSE),

  data = model_data,
  prior = priors_mediation,

  # MCMC settings
  chains = 4,           # Number of Markov chains
  iter = 4000,          # Total iterations per chain
  warmup = 2000,        # Warmup (burn-in) iterations
  cores = 4,            # Parallel processing
  seed = 42,            # Reproducibility

  # Computational settings
  control = list(
    adapt_delta = 0.95,   # Increase if divergent transitions
    max_treedepth = 12    # Increase if max treedepth warnings
  )
)

In [ ]:
# View model summary
summary(fit_mediation)

## Interpretation Guide

The model summary shows:

### Fixed Effects Table

| Column | Meaning |
|--------|--------|
| Estimate | Posterior median |
| Est.Error | Posterior standard deviation |
| l-95% CI | Lower bound of 95% credible interval |
| u-95% CI | Upper bound of 95% credible interval |
| Rhat | Convergence diagnostic (want < 1.01) |
| Bulk_ESS | Effective sample size for mean |
| Tail_ESS | Effective sample size for tails |

In [ ]:
# Extract fixed effects
fixef_table <- fixef(fit_mediation)
print(round(fixef_table, 3))

**Interpretation of key paths:**

- `TTM_z_BNR_z`: Path $a_1$ (main effect of X on M)
- `TTM_z_BNR_z:FP_z`: Path $a_3$ (moderation of X→M by W)
- `FS_TTM_z`: Path $b$ (effect of M on Y)
- `FS_BNR_z`: Path $c'$ (direct effect of X on Y)

---

# Step 5: Check Diagnostics

## Conceptual Explanation

Before trusting your results, verify that the MCMC sampler converged properly:

1. **Rhat**: Measures between-chain vs. within-chain variance (want < 1.01)
2. **ESS**: Effective sample size accounting for autocorrelation (want > 400)
3. **Trace plots**: Visual check for chain mixing
4. **Posterior predictive checks**: Does the model reproduce data patterns?

## Code Snippet

In [ ]:
# Check Rhat and ESS for all parameters
rhat_values <- rhat(fit_mediation)
ess_values <- neff_ratio(fit_mediation) * 8000  # Total post-warmup samples

cat("=== Convergence Diagnostics ===\n")
cat("Max Rhat:", round(max(rhat_values), 4), "(should be < 1.01)\n")
cat("Min ESS:", round(min(ess_values)), "(should be > 400)\n")
cat("All Rhat < 1.01:", all(rhat_values < 1.01), "\n")
cat("All ESS > 400:", all(ess_values > 400), "\n")

In [ ]:
# Trace plots for key parameters
plot(fit_mediation, variable = c("b_TTMz_BNR_z", "b_FS_TTM_z"),
     N = 3, ask = FALSE)

In [ ]:
# Posterior predictive check for mediator
pp_check(fit_mediation, resp = "TTMz", ndraws = 100) +
  labs(title = "Posterior Predictive Check: TTM")

## Interpretation Guide

### Rhat Interpretation

| Rhat Value | Interpretation | Action |
|------------|----------------|--------|
| < 1.01 | Converged | Proceed |
| 1.01 - 1.05 | Marginal | Run longer or check model |
| > 1.05 | Not converged | Do NOT trust results |

### Posterior Predictive Check Interpretation

- **Good**: Simulated data (light lines) overlap observed data (dark line)
- **Bad**: Systematic differences suggest model misspecification

---

# Step 6: Compute Indirect Effects

## Conceptual Explanation

The indirect effect is the product of paths: $\text{Indirect} = a \times b$

For moderated mediation, the indirect effect varies with the moderator:

$$\text{Indirect}(W) = (a_1 + a_3 \times W) \times b$$

Where:
- $a_1$ = main effect of X on M
- $a_3$ = X × W interaction effect on M
- $b$ = effect of M on Y

### Index of Moderated Mediation (IMM)

The IMM quantifies how much the indirect effect changes per unit change in W:

$$\text{IMM} = a_3 \times b$$

If IMM's credible interval excludes zero, moderated mediation is present.

## Code Snippet

In [ ]:
library(posterior)

# Extract posterior samples
posts <- as_draws_df(fit_mediation)

# Extract path coefficients
a1 <- posts$b_TTMz_BNR_z           # BNR -> TTM (main effect)
a3 <- posts$`b_TTMz_BNR_z:FP_z`    # BNR × FP -> TTM (interaction)
b <- posts$b_FS_TTM_z              # TTM -> FS
c_prime <- posts$b_FS_BNR_z        # BNR -> FS (direct)

# Compute indirect effects at different levels of moderator
indirect_low <- (a1 + a3 * (-1)) * b    # FP at -1 SD
indirect_mean <- a1 * b                  # FP at mean (0)
indirect_high <- (a1 + a3 * 1) * b      # FP at +1 SD

# Index of Moderated Mediation
IMM <- a3 * b

# Total effect
total <- c_prime + indirect_mean

# Create summary table
mediation_summary <- tibble(
  Effect = c("a1 (BNR → TTM)",
             "a3 (BNR×FP → TTM)",
             "b (TTM → FS)",
             "c' (direct)",
             "Indirect (low FP)",
             "Indirect (mean FP)",
             "Indirect (high FP)",
             "IMM",
             "Total effect"),
  Estimate = c(median(a1), median(a3), median(b), median(c_prime),
               median(indirect_low), median(indirect_mean),
               median(indirect_high), median(IMM), median(total)),
  CI_lower = c(quantile(a1, 0.025), quantile(a3, 0.025),
               quantile(b, 0.025), quantile(c_prime, 0.025),
               quantile(indirect_low, 0.025), quantile(indirect_mean, 0.025),
               quantile(indirect_high, 0.025), quantile(IMM, 0.025),
               quantile(total, 0.025)),
  CI_upper = c(quantile(a1, 0.975), quantile(a3, 0.975),
               quantile(b, 0.975), quantile(c_prime, 0.975),
               quantile(indirect_low, 0.975), quantile(indirect_mean, 0.975),
               quantile(indirect_high, 0.975), quantile(IMM, 0.975),
               quantile(total, 0.975)),
  Prob_positive = c(mean(a1 > 0), mean(a3 > 0), mean(b > 0), mean(c_prime > 0),
                    mean(indirect_low > 0), mean(indirect_mean > 0),
                    mean(indirect_high > 0), mean(IMM > 0), mean(total > 0))
)

print(mediation_summary)

In [ ]:
# Visualize indirect effect distribution
indirect_df <- tibble(
  `Low FP (-1 SD)` = indirect_low,
  `Mean FP (0)` = indirect_mean,
  `High FP (+1 SD)` = indirect_high
) %>%
  pivot_longer(everything(), names_to = "Condition", values_to = "Indirect")

ggplot(indirect_df, aes(x = Indirect, fill = Condition)) +
  geom_density(alpha = 0.5) +
  geom_vline(xintercept = 0, linetype = "dashed") +
  labs(title = "Posterior Distribution of Indirect Effects",
       x = "Indirect Effect", y = "Density") +
  theme_minimal() +
  scale_fill_brewer(palette = "Set2")

## Interpretation Guide

### Reading the Results Table

| Effect | Interpretation |
|--------|---------------|
| Estimate | Posterior median (central tendency) |
| CI_lower, CI_upper | 95% credible interval |
| Prob_positive | Posterior probability effect > 0 |

### Decision Rules

1. **Significant indirect effect**: CI excludes zero
2. **Moderated mediation present**: IMM CI excludes zero
3. **Full mediation**: Direct effect (c') CI includes zero while indirect doesn't
4. **Partial mediation**: Both direct and indirect CIs exclude zero

---

# Step 7: Test Moderation

## Conceptual Explanation

Moderation testing involves:

1. **Testing the interaction term**: Is $a_3$ (or $b_3$) different from zero?
2. **Simple slopes analysis**: What is the effect of X on M at different levels of W?
3. **Regions of significance**: At what values of W is the effect significant?

## Code Snippet

In [ ]:
# Test interaction effect using hypothesis()
cat("=== Testing Moderation (BNR × FP interaction) ===\n\n")

# Bayesian hypothesis test
h_interaction <- hypothesis(fit_mediation, "TTMz_BNR_z:FP_z = 0")
print(h_interaction)

# Manual calculation with posterior
cat("\n--- Interaction Effect Summary ---\n")
cat("Median:", round(median(a3), 4), "\n")
cat("95% CI: [", round(quantile(a3, 0.025), 4), ",",
    round(quantile(a3, 0.975), 4), "]\n")
cat("P(a3 != 0):", round(mean(abs(a3) > 0.001), 4), "\n")
cat("P(a3 < 0):", round(mean(a3 < 0), 4), "\n")

In [ ]:
# Simple slopes: Effect of BNR on TTM at different FP levels
FP_levels <- c(-1, 0, 1)  # Low, Mean, High

simple_slopes <- map_dfr(FP_levels, function(fp) {
  slope <- a1 + a3 * fp
  tibble(
    FP_level = fp,
    FP_label = case_when(
      fp == -1 ~ "Low (-1 SD)",
      fp == 0 ~ "Mean",
      fp == 1 ~ "High (+1 SD)"
    ),
    Slope = median(slope),
    CI_lower = quantile(slope, 0.025),
    CI_upper = quantile(slope, 0.975),
    Prob_negative = mean(slope < 0)
  )
})

cat("\n=== Simple Slopes Analysis ===\n")
cat("Effect of BNR on TTM at different levels of FP:\n\n")
print(simple_slopes)

In [ ]:
# Visualize simple slopes
slopes_for_plot <- expand_grid(
  BNR_z = seq(-2, 2, length.out = 100),
  FP_level = c(-1, 0, 1)
) %>%
  mutate(
    FP_label = factor(FP_level, levels = c(-1, 0, 1),
                      labels = c("Low FP (-1 SD)", "Mean FP", "High FP (+1 SD)")),
    # Predicted TTM (using posterior medians for visualization)
    TTM_pred = median(posts$b_TTMz_Intercept) +
               median(a1) * BNR_z +
               median(posts$b_TTMz_FP_z) * FP_level +
               median(a3) * BNR_z * FP_level
  )

ggplot(slopes_for_plot, aes(x = BNR_z, y = TTM_pred, color = FP_label)) +
  geom_line(linewidth = 1.2) +
  labs(title = "Simple Slopes: BNR Effect on TTM by Founder Preparedness",
       x = "Broker Network Richness (standardized)",
       y = "Time to Meeting (standardized)",
       color = "Moderator Level") +
  theme_minimal() +
  scale_color_brewer(palette = "Set1")

In [ ]:
# Johnson-Neyman: Region of significance
# At what value of FP does the BNR effect become significant?

# Calculate effect of BNR at many FP values
FP_range <- seq(-3, 3, by = 0.1)

JN_results <- map_dfr(FP_range, function(fp) {
  slope <- a1 + a3 * fp
  tibble(
    FP = fp,
    Effect = median(slope),
    CI_lower = quantile(slope, 0.025),
    CI_upper = quantile(slope, 0.975),
    Significant = (CI_lower > 0) | (CI_upper < 0)
  )
})

# Plot Johnson-Neyman
ggplot(JN_results, aes(x = FP, y = Effect)) +
  geom_ribbon(aes(ymin = CI_lower, ymax = CI_upper),
              fill = "steelblue", alpha = 0.3) +
  geom_line(color = "steelblue", linewidth = 1) +
  geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
  geom_rug(data = filter(JN_results, Significant),
           aes(x = FP), sides = "b", color = "darkgreen") +
  labs(title = "Johnson-Neyman Plot: Regions of Significance",
       subtitle = "Green ticks indicate FP values where BNR effect is significant",
       x = "Founder Preparedness (standardized)",
       y = "Effect of BNR on TTM") +
  theme_minimal()

---

# Step 8: Report Results

## Conceptual Explanation

A complete report includes:

1. **Method description**: Model specification, priors, software
2. **Convergence evidence**: Rhat, ESS, diagnostics
3. **Path coefficients**: With credible intervals
4. **Mediation results**: Indirect effects, IMM
5. **Visualizations**: Path diagram, simple slopes
6. **Interpretation**: In context of hypotheses

## Code Snippet: Results Table

In [ ]:
# Create publication-ready table
results_table <- tibble(
  Path = c("BNR → TTM (a₁)",
           "BNR × FP → TTM (a₃)",
           "TTM → FS (b)",
           "BNR → FS (c')",
           "---",
           "Indirect (mean FP)",
           "IMM",
           "Total Effect"),
  Estimate = c(sprintf("%.3f", median(a1)),
               sprintf("%.3f", median(a3)),
               sprintf("%.3f", median(b)),
               sprintf("%.3f", median(c_prime)),
               "---",
               sprintf("%.3f", median(indirect_mean)),
               sprintf("%.3f", median(IMM)),
               sprintf("%.3f", median(total))),
  `95% CI` = c(sprintf("[%.3f, %.3f]", quantile(a1, 0.025), quantile(a1, 0.975)),
               sprintf("[%.3f, %.3f]", quantile(a3, 0.025), quantile(a3, 0.975)),
               sprintf("[%.3f, %.3f]", quantile(b, 0.025), quantile(b, 0.975)),
               sprintf("[%.3f, %.3f]", quantile(c_prime, 0.025), quantile(c_prime, 0.975)),
               "---",
               sprintf("[%.3f, %.3f]", quantile(indirect_mean, 0.025), quantile(indirect_mean, 0.975)),
               sprintf("[%.3f, %.3f]", quantile(IMM, 0.025), quantile(IMM, 0.975)),
               sprintf("[%.3f, %.3f]", quantile(total, 0.025), quantile(total, 0.975))),
  `P(Direction)` = c(sprintf("%.1f%%", mean(a1 < 0) * 100),
                     sprintf("%.1f%%", max(mean(a3 < 0), mean(a3 > 0)) * 100),
                     sprintf("%.1f%%", mean(b < 0) * 100),
                     sprintf("%.1f%%", mean(c_prime > 0) * 100),
                     "---",
                     sprintf("%.1f%%", mean(indirect_mean > 0) * 100),
                     sprintf("%.1f%%", max(mean(IMM > 0), mean(IMM < 0)) * 100),
                     sprintf("%.1f%%", mean(total > 0) * 100))
)

print(results_table)

## Writing Results (Template)

> **Mediation Results**: We tested whether Time to Meeting (TTM) mediated the relationship between Broker Network Richness (BNR) and Funding Success (FS) using Bayesian moderated mediation analysis. All models were estimated using brms (Bürkner, 2017) with weakly informative priors (Normal(0, 1.5) for fixed effects). Convergence was confirmed by Rhat < 1.01 and ESS > 400 for all parameters.
>
> The a-path from BNR to TTM was [coefficient], 95% CI [lower, upper], indicating that founders with more diverse broker networks obtained investor meetings faster. The b-path from TTM to FS was [coefficient], 95% CI [lower, upper], confirming that faster meetings predicted higher funding success.
>
> The indirect effect at mean founder preparedness was [coefficient], 95% CI [lower, upper], supporting mediation. The Index of Moderated Mediation was [IMM coefficient] (95% CI [lower, upper]), with the credible interval [excluding/including] zero, indicating that founder preparedness [significantly/did not significantly] moderates the mediation pathway.

---

# Appendix: Complete Analysis Template

In [ ]:
# ================================================================
# COMPLETE BAYESIAN MODERATED MEDIATION TEMPLATE
# ================================================================

# --- Setup ---
library(brms)
library(tidyverse)
library(posterior)

# --- Step 1: Define Model ---
# X -> M -> Y with W moderating X -> M (first-stage)

# --- Step 2: Prepare Data ---
model_data <- raw_data %>%
  mutate(
    X_z = scale(X)[,1],
    W_z = scale(W)[,1],
    M_z = scale(M)[,1]
  ) %>%
  drop_na()

# --- Step 3: Specify Priors ---
priors <- c(
  prior(normal(0, 1.5), class = "b", resp = "Mz"),
  prior(normal(0, 1.5), class = "b", resp = "Y"),
  prior(exponential(1), class = "sd", resp = "Mz"),
  prior(exponential(1), class = "sd", resp = "Y"),
  prior(exponential(1), class = "sigma", resp = "Mz")
)

# --- Step 4: Fit Model ---
fit <- brm(
  bf(M_z ~ X_z * W_z + (1 | Group)) +
  bf(Y ~ X_z + W_z + M_z + (1 | Group), family = bernoulli()) +
  set_rescor(FALSE),
  data = model_data,
  prior = priors,
  chains = 4, iter = 4000, warmup = 2000, cores = 4, seed = 42,
  control = list(adapt_delta = 0.95)
)

# --- Step 5: Check Diagnostics ---
stopifnot(max(rhat(fit)) < 1.01)
stopifnot(min(neff_ratio(fit)) > 0.1)

# --- Step 6: Compute Indirect Effects ---
posts <- as_draws_df(fit)
a1 <- posts$b_Mz_X_z
a3 <- posts$`b_Mz_X_z:W_z`
b <- posts$b_Y_M_z

indirect_mean <- a1 * b
IMM <- a3 * b

# --- Step 7: Test Moderation ---
simple_slopes <- map_dfr(c(-1, 0, 1), ~tibble(
  W = .x,
  slope = median(a1 + a3 * .x),
  CI = list(quantile(a1 + a3 * .x, c(0.025, 0.975)))
))

# --- Step 8: Report Results ---
cat("Indirect effect:", median(indirect_mean),
    "95% CI:", quantile(indirect_mean, c(0.025, 0.975)), "\n")
cat("IMM:", median(IMM),
    "95% CI:", quantile(IMM, c(0.025, 0.975)), "\n")

---

# Step 9: Simulate to Understand Your Power

## Conceptual Explanation

Before collecting data---or before interpreting a null result---you need to know: *Would my study have detected moderated mediation if it existed?*

This step walks through a simulation-based power analysis. The idea is simple: generate fake data where you *know* the true IMM, fit your model, and see how often you detect it. Do this 500 times, and you have your power estimate.

### What We're Testing

We vary three things:

1. **Sample size**: How many participants do you need?
2. **Effect size**: How strong is the "true" moderation effect?
3. **Missing data**: Does attrition hurt your power?

### Simulation Hypotheses

Based on Bayesian estimation theory and prior simulation work on mediation, we specify three confirmatory hypotheses:

- **H1 (Unbiased Estimation)**: Bayesian estimation will recover true IMM values with minimal bias (|bias| < 0.01) across all conditions.
- **H2 (Nominal Coverage)**: The 95% credible intervals will achieve nominal coverage rates (94--96%) when model assumptions are satisfied.
- **H3 (Adequate Power)**: Power will reach the conventional threshold of 80% for medium effects (IMM ≈ 0.10) at sample sizes of N ≥ 250.

## Simulation Results Summary

### The Bottom Line

**How many participants do you need?**

| Your Expected IMM | Minimum N for 80% Power |
|-------------------|------------------------|
| Large (0.16) | **100** will do it |
| Medium (0.10) | Need at least **250** |
| Small (0.04) | You'll need **500-1000** |

---

# Step 10: Apply to Real Data (Four Examples)

## Conceptual Explanation

We walk through **four real datasets** from different research areas:

- Two show **significant** moderated mediation (it worked!)
- Two show **null** results (it didn't work---and that's informative too)

## The Four Datasets at a Glance

| Dataset | Domain | N | Model | Verdict |
|---------|--------|---|-------|--------|
| Garcia (Protest/Sexism) | Social Psychology | 129 | Protest → Response → Liking \| Sexism | ✓ Significant |
| SLID (Labor Economics) | Labor Economics | 2000 | Age → Education → Wages \| Sex | ✓ Significant |
| Tal-Or (Media Influence) | Communication | 123 | Placement → PMI → Behavior \| Importance | ✗ Null |
| FacialBurns (Clinical) | Health Psychology | 98 | Severity → Distress → Self-Esteem \| Rumination | ✗ Null |

## Example 1: Garcia Protest Dataset ✓ SIGNIFICANT

### The Research Question

Does protesting workplace discrimination make people like you more? And does it depend on whether observers believe sexism is real?

**Model**: Protest → Perceived Appropriateness → Liking

**Moderator**: Perceived Sexism (first-stage)

### The Results

**IMM = 0.27 [0.07, 0.49]** --- the interval excludes zero!

*Translation*: The mediation pathway is **stronger** when observers score high on perceived sexism. Among people who recognize sexism as a problem, protesting is perceived as more appropriate, which is associated with higher liking ratings.

## Example 2: SLID Labor Economics Dataset ✓ SIGNIFICANT

### The Research Question

Does education mediate the relationship between age and wages? Does this differ for men vs. women?

**Model**: Age → Education → Wages

**Moderator**: Sex (first-stage)

### The Results

**IMM = 0.20 [0.12, 0.28]** --- clearly excludes zero.

The story here is about **cohort effects**. Among women, older = less education = lower wages. Among men, this effect nearly disappears.

## Example 3: Tal-Or Media Dataset ✗ NULL RESULT

**IMM = 0.02 [-0.06, 0.11]** --- includes zero. No evidence of moderated mediation.

## Example 4: FacialBurns Clinical Dataset ✗ NULL RESULT

**IMM = -0.05 [-0.19, 0.07]** --- includes zero.

---

# Step 11: Bayesian vs. Frequentist---Which Should You Use?

## Conceptual Explanation

You've got two options for moderated mediation: Bayesian (this tutorial) or frequentist (PROCESS macro, lavaan bootstrap). How do they compare?

**Spoiler**: For most well-powered studies, they give nearly identical answers. The differences emerge at the edges---small samples, complex models, and how you interpret your results.

## When Do They Differ?

### Scenario 1: Small Samples (N < 100)

At N = 50, frequentist bootstrap **under-covers**---only 89% of "95% CIs" contain the true value. The Bayesian priors act as regularizers, keeping coverage at ~94%.

### Scenario 2: The Interpretation Difference

| What You Want to Say | Bayesian CrI | Frequentist CI |
|---------------------|--------------|----------------|
| "There's a 95% chance the IMM is between 0.07 and 0.49" | **Correct** ✓ | Incorrect ✗ |
| "If we repeated this 100 times, ~95 intervals would contain the true IMM" | Correct ✓ | **Correct** ✓ |

Most researchers *want* to make the first statement. Only Bayesian intervals allow it.

## Decision Flowchart

```
Should you use Bayesian or Frequentist?
                    |
        Is N < 100? ----YES----> Use Bayesian
                    |                (better coverage)
                   NO
                    |
        Is Y binary? ---YES----> Use Bayesian
                    |                (handles mixed families)
                   NO
                    |
    Do you want posterior ----YES----> Use Bayesian
    probabilities? (P(IMM > 0))        (only option)
                    |
                   NO
                    |
           Either works fine!
    (Results will be nearly identical)
```

## The Bottom Line

| If you... | Use... | Why |
|-----------|--------|-----|
| Have N > 200, simple model | Either | Results converge |
| Have N < 100 | Bayesian | Better coverage |
| Have binary Y | Bayesian | Handles mixed families |
| Want P(IMM > 0) | Bayesian | Only option |
| Need speed (100+ models) | Frequentist | Faster bootstrap |
| Have skeptical reviewers | Frequentist | More familiar |

**Our recommendation**: Default to Bayesian. The flexibility, proper uncertainty quantification, and intuitive interpretation outweigh the computational cost for most research applications.

---

# Limitations and Boundary Conditions

While Bayesian moderated mediation offers substantial advantages, researchers should be aware of contexts where it may not be optimal or appropriate.

## When NOT to Use This Approach

**1. Extremely large-sample studies (N > 10,000)**

With very large samples, the computational overhead of MCMC sampling provides negligible benefit over frequentist bootstrapping.

**2. Real-time or high-throughput applications**

If you need to fit hundreds of models quickly, MCMC is too slow.

**3. Contexts requiring exact frequentist properties**

Some regulatory contexts (e.g., FDA clinical trials) require pre-registered frequentist analyses.

## Causal Interpretation Caveats

Moderated mediation is an inherently causal framework, but cross-sectional data cannot establish causality. Even with longitudinal data, unobserved confounding can bias path coefficients. The methods in this tutorial estimate conditional associations; causal claims require additional design features (randomization, instrumental variables, etc.).

---

# Software and Data Availability

## Software Versions

All analyses were conducted in R version 4.3.1 with the following key packages:

| Package | Version | Purpose |
|---------|---------|--------|
| brms | 2.20.4 | Bayesian regression modeling |
| rstan | 2.32.2 | Stan interface |
| posterior | 1.5.0 | Posterior processing |
| tidyverse | 2.0.0 | Data manipulation |
| lavaan | 0.6-16 | Frequentist SEM comparison |

## Code and Data Repository

Complete R code for all analyses is available at:

- **GitHub**: https://github.com/onionbryan/baysean

The repository includes:

- `R/` - All code chunks from this document
- `simulation_functions.R` - Parallel simulation framework
- `empirical_analysis/` - Scripts for each dataset
- `prior_sensitivity.R` - Sensitivity analysis code
- `figures/` - Publication-ready figures

**Data sources:**

- **Garcia protest dataset**: Available in R package `processR`
- **SLID dataset**: Available in R package `carData`
- **Tal-Or media dataset**: Available in R package `processR`
- **FacialBurns dataset**: Simulated based on published summary statistics

---

# Author Contributions (CRediT)

**Bryan Gibson**: Conceptualization (lead), Methodology (lead), Software (lead), Formal Analysis (lead), Writing – Original Draft (lead), Writing – Review & Editing (lead), Visualization (lead).

## Acknowledgments

The author thanks the anonymous reviewers for their constructive feedback and Paul Bürkner for developing and maintaining the brms package. This work was supported in part by resources from the University of Southern Indiana.

## Conflict of Interest

The author declares no conflicts of interest.

---

# References

Bürkner, P. C. (2017). brms: An R package for Bayesian multilevel models using Stan. *Journal of Statistical Software*, 80(1), 1-28.

Carpenter, B., Gelman, A., Hoffman, M. D., Lee, D., Goodrich, B., Betancourt, M., ... & Riddell, A. (2017). Stan: A probabilistic programming language. *Journal of Statistical Software*, 76(1), 1-32.

Gelman, A., Jakulin, A., Pittau, M. G., & Su, Y. S. (2008). A weakly informative default prior distribution for logistic and other regression models. *Annals of Applied Statistics*, 2(4), 1360-1383.

Hayes, A. F. (2013). *Introduction to mediation, moderation, and conditional process analysis: A regression-based approach*. Guilford Press.

Hayes, A. F. (2018). *Introduction to mediation, moderation, and conditional process analysis: A regression-based approach* (2nd ed.). Guilford Press.

Hayes, A. F., & Rockwood, N. J. (2017). Regression-based statistical mediation and moderation analysis in clinical research: Observations, recommendations, and implementation. *Behaviour Research and Therapy*, 98, 39-57.

Kruschke, J. K. (2015). *Doing Bayesian data analysis: A tutorial with R, JAGS, and Stan* (2nd ed.). Academic Press.

MacKinnon, D. P. (2008). *Introduction to statistical mediation analysis*. Erlbaum.

McElreath, R. (2020). *Statistical rethinking: A Bayesian course with examples in R and Stan* (2nd ed.). CRC Press.

McNeish, D. (2016). On using Bayesian methods to address small sample problems. *Structural Equation Modeling*, 23(5), 750-773.

Miočević, M., O'Rourke, H. P., MacKinnon, D. P., & Brown, H. C. (2017). Statistical properties of four effect-size measures for mediation models. *Behavior Research Methods*, 50(1), 285-301.

Muthén, B., & Asparouhov, T. (2012). Bayesian structural equation modeling: A more flexible representation of substantive theory. *Psychological Methods*, 17(3), 313-335.

Preacher, K. J., & Hayes, A. F. (2008). Asymptotic and resampling strategies for assessing and comparing indirect effects in multiple mediator models. *Behavior Research Methods*, 40(3), 879-891.

Preacher, K. J., Rucker, D. D., & Hayes, A. F. (2007). Addressing moderated mediation hypotheses: Theory, methods, and prescriptions. *Multivariate Behavioral Research*, 42(1), 185-227.

Smid, S. C., McNeish, D., Miočević, M., & van de Schoot, R. (2020). Bayesian versus frequentist estimation for structural equation models in small sample contexts: A systematic review. *Structural Equation Modeling*, 27(1), 131-161.

Wang, L., & Preacher, K. J. (2015). Moderated mediation analysis using Bayesian methods. *Structural Equation Modeling*, 22(2), 249-263.

Yuan, Y., & MacKinnon, D. P. (2009). Bayesian mediation analysis. *Psychological Methods*, 14(4), 301-322.